In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import random
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with gzip.open('train_cases.pkl.gz','rb') as fp:
    train_cases = pickle.load(fp)
type(train_cases), len(train_cases)

(tuple, 2)

In [3]:
train_X = train_cases[0]
train_y = train_cases[1]
ntrain = train_X.shape[0]
type(train_X), train_X.shape, train_X.dtype, type(train_y), train_y.shape, train_y.dtype

(numpy.ndarray,
 (16384, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (16384, 1),
 dtype('float64'))

In [4]:
with gzip.open('valid_cases.pkl.gz','rb') as fp:
    valid_cases = pickle.load(fp)
valid_X = valid_cases[0]
valid_y = valid_cases[1]
nvalid = valid_X.shape[0]
type(valid_X), valid_X.shape, valid_X.dtype, type(valid_y), valid_y.shape, valid_y.dtype

(numpy.ndarray,
 (4096, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (4096, 1),
 dtype('float64'))

In [5]:
def choose_training_case():
    i = random.randint(0, ntrain-1)
    return(train_X[i,:,:,:], train_y[i,:])

def choose_validation_case():
    i = random.randint(0, nvalid-1)
    return(valid_X[i,:,:,:], valid_y[i,:])

In [6]:
def make_batch_from_megabatch(batchsize=32, validation=False):
    get_case = choose_validation_case if validation else choose_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        case = get_case()
        x = np.expand_dims(case[0], axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(case[1])).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(xbatch, ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=True))

In [7]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [8]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [354]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, input_shape=input_shape)) # 64
#model.add(keras.layers.Conv2D(32, 1, input_shape=input_shape))
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32, 1))  # 32
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Conv2D(16, 1, activation='relu'))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(16, (2,4))) # 16
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0.0))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(25, (4,6))) # 29,5
#model.add(keras.layers.Conv2D(25, 5)) # 29,5
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0.0))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(4, (2,4))) # 4,3
#model.add(keras.layers.Conv2D(4, 3)) # 4,3
model.add(keras.layers.LeakyReLU(0.0))
#model.add(keras.layers.PReLU())
#model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Conv2D(3, 5))
#model.add(keras.layers.LeakyReLU(0))
#model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Flatten())
#model.add(keras.layers.Dropout(.3)) # or .2
#model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dropout(.5)) # .5 or .6
#model.add(keras.layers.GaussianDropout(.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [355]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [356]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_341 (Conv2D)          (None, 16, 16, 64)        8256      
_________________________________________________________________
leaky_re_lu_341 (LeakyReLU)  (None, 16, 16, 64)        0         
_________________________________________________________________
batch_normalization_273 (Bat (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_342 (Conv2D)          (None, 16, 16, 32)        2080      
_________________________________________________________________
leaky_re_lu_342 (LeakyReLU)  (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_274 (Bat (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_343 (Conv2D)          (None, 15, 13, 16)        4112      
__________

In [357]:
valid=valid_gen()
train=train_gen()
model.fit_generator(train, 
                    validation_data=valid, validation_steps=64,
                    epochs=60, 
                    steps_per_epoch=64)

Epoch 1/60
64/64 [==============================] - 8s 129ms/step - loss: 0.6457 - acc: 0.6992 - val_loss: 0.5836 - val_acc: 0.7485
Epoch 2/60
64/64 [==============================] - 1s 18ms/step - loss: 0.5673 - acc: 0.7544 - val_loss: 0.5524 - val_acc: 0.7607
Epoch 3/60
64/64 [==============================] - 1s 18ms/step - loss: 0.5510 - acc: 0.7695 - val_loss: 0.5460 - val_acc: 0.7720
Epoch 4/60
64/64 [==============================] - 1s 18ms/step - loss: 0.5582 - acc: 0.7578 - val_loss: 0.5514 - val_acc: 0.7578
Epoch 5/60
64/64 [==============================] - 1s 18ms/step - loss: 0.5403 - acc: 0.7646 - val_loss: 0.5200 - val_acc: 0.7749
Epoch 6/60
64/64 [==============================] - 1s 18ms/step - loss: 0.5292 - acc: 0.7646 - val_loss: 0.5293 - val_acc: 0.7568
Epoch 7/60
64/64 [==============================] - 1s 18ms/step - loss: 0.5102 - acc: 0.7681 - val_loss: 0.5428 - val_acc: 0.7505
Epoch 8/60
64/64 [==============================] - 1s 18ms/step - loss: 0.4716 - 

In [358]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(c, axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

EternalFlame 115
Predicted:  0.015809983    Actual:  False 

EternalFlame 115
Predicted:  0.020323075    Actual:  False 

SmoothOperator 48
Predicted:  8.087957e-06    Actual:  False 

ChatanoogaChooChoo 42
Predicted:  0.10401919    Actual:  False 

ChatanoogaChooChoo 235
Predicted:  0.6460627    Actual:  False 

HoldMeTight 272
Predicted:  0.7731308    Actual:  True 

IfIHadAHammer 75
Predicted:  0.029689062    Actual:  False 

InTheHillsOfShiloh 75
Predicted:  0.4024067    Actual:  False 

ChatanoogaChooChoo 75
Predicted:  0.00031394788    Actual:  False 

Kalamazoo 68
Predicted:  0.5599123    Actual:  True 



<code>
from 4f baseline:
3x3 conv to 2x4    -> 2809, 3087, 3104, 2887, 2767, 2852, 2763, 3067, 2994, 3153, 3011,
                      **2929**, 2777, 2745, 2806, 3070, 3197
above + 5x5 to 4x6 -> 3163, 2705, 2931, **2829**, 2915, 3111, 2765, 2791, 3152, 2813, 2797,
                      2771, *2836*, 2876, 3039, 2951, 2726, 3144, 2734, 2699, 2732,
                      2968, 2755, 3082, 2881, 2723, 2809, 3320, 2659, 2936, *2814*
above + 5x5 to 6x4 -> 2871, 3178
3x3 conv to 4x2 -> 3297, 3458
*2x4, 4x6, 2x4      -> 2728, 2940, 3030, *2821*, 2440, 2911, 2914, 2924, 3134, 2708, 2534,
                      2910, 3352, 2870, 2790, **2816**, 2735, 2763, 2767, 2942, 3030,
                      2786, 2942, 2772, 3089, 2777, 2854, 2690, 2912, 2659, 2799,
                      2770, *2801* *
    
from 5f baseline:
drop bottom b/n -> 3050, 3032, 2667, 3353, 2736, **3132**, 3210, 3199, 3359
drop top b/n -> 3266, 2870, 3306, 3039, 2917, **3201**, **3040**, 3304
60-25-50-20-5 -> 3237, 3136
</code><code>`
80-50-20-32-5 -> 2939, **3086**, 2849, 3110, 2990, 3200, 3111
50-20-14-25-4 -> 3379
80-50-16-25-4 -> 3165, **3079**, 2868, 2994, 3093, 2995, 2951, 3268, 3145
</code>

<code>
with ,5 baseline:
activate after b/n -> 3054, 3095, 3618, 3548, **3338**
leaky .1 -> 3149
prelu -> 3080, 3609, 3620
prelu d/o=.6 -> 3734
1-1-3-5-3-1 -> 2801, 3374, 3578, 3305, 3015
gaussian d/o -> 2911, 3217, 2953, 3097, 3095, 3376, **3212**, 3427, 3706
1-1-3-5-5 -> 3215, 3180
leaky .2 on bottom 2 layers -> 2725, 3098, 2977, 3094, 2863, 3073, 3206, 3320, 3036, 
                             **3077**, 3217 SOMETIMES d/o=.6
leaky .2 on bottom 2 layers -> 3189, 2989, 3454, 3088, 2928, 3036, **3094**, 3500, 3238,
                               3194, 3073, 3066, 3357
leaky .4 on bottom 2 layers -> 3248
leaky .2/.1 on bottom layers -> 2839, 2832, **3135**, 2908, 3350, 3075, 3161, 3342, 3247
1-1-1-5-5 -> 3315, 3229
16 filters to 32 -> 3472
delete bottom layer -> 3820
extra 16 layer -> 3312
8 filters to 32 -> **3174**, 2941, 3190
8,4 filters to 25,5 -> 2801, 3542, 2648, 3495, 2957, 3255, 2841, 2843, 2863, 2701, 3681,
                       3207, 3480, 3262, 2864, 2839, 3056, **3029**, 2928, 3086, 3240,
                       2792, 2899, 3097, 2880, 2969, 3049, 3004, 2736, 3621,
                       3090, 3305, 2955, 3249, 3193
    
above w .65 dropout -> 2924, 2897, 3077, 2902, 3279, 3015, 2854, 2893, 3157, 2984, 3127,
                        2984, 3130, 3094, 3232, 2779, **3043**, 3188, 2983, 3228, 3294
above w .75 dropout -> 2908, 3218, 2805, **3047**, 3333, **3081**
8,4 filter to 25, 3 -> 3439
8,4 filter to 25, 6 -> 3236, 2872, 3313
*8,4 filter to 25, 4 -> 2801, 2983, 3097, 2959, 2899, 2939, 3295, 3111, 3105, 2902, 3140,
                        2885, 2806, 3087, 2928, 2993, 2901, 3070, 3115, 3057, **2988**,
                        2993, 3547, 3287, 2936, 2953, 2873, 3312, 3089, 2983, 2812*
                        
                        
8,4 f to 29,4, d/o=.6 -> 2937, 2845, 3183, 3478, 3002, 3139, 2927, 2961, 3560, 2921, 3266,
                         **3113**, 3163 

16,8,4 f to 12,25,4 -> 3170, 2928, 3089, 3339, **3166**
</code><code>                        
add (3,3) layer above, d/o=.5 -> 3181
above w d/o=.65 -> 2962, 3142, 2984, 3086, 2914, **3048**, 3625
(3,5) layer instead -> 3330
above w .5 d/o -> 3290
extra 50 dense layer (30/60 d/o) -> 2954, 3378, 2968, 3393, **3113**
above w 20/50 dropout -> 3375

used .6 baseline
leaky .05 -> 3122, 3245
</code>

<code>
1-1-5-5-5 -> 3107, 3481 (median of last 5)
1-1-3-5-3 -> 3052, 3053, 2858, 3295, 3636, 2974, 3057, 2834, 3045, 3742, 3182, 
             3559, 3112, **3104**, 3148, 4013, 3305, 3015
1-1-3-5-3-1 -> 3064, 3129, 3062, 3237
1-1-3-5-3-1 d/o=.4 -> 2820, 3224, 3211
1-1-3-5-3 d/o=.45 -> 3215
1-1-3-5-3 d/o=.7 -> 3408
1-1-3-5-3-5 -> 3625
*1-1-3-5-3 d/o=.5 -> 3021, 3100, 2908, 3027, 2970, 3188, 3044, 3055, 3021, 3143, 3278, 
                     3378, 3213, 3058, 2873, 2788, 3144, 2777, 3248, 2975, 3244, 
                     3219, 3570, 3374, 3118, **3082**, 3039 * 
1-1-3-5-3-3 d/o=.5 -> 3151, 3149, 3533
1-1-3-5-3 add maxpool d/o=.5 -> 3193
1-1-3-5-3 add maxpool d/o=.65 -> 4213
1-1-1-3-5-3 d/o=.5 -> 3204, 
</code>